In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col

# Set the location of the Delta Lake and Kafka packages
delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"  # Replace with the correct Spark version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"

# Initialize Spark Session with Delta Lake and Kafka support
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-28940b78-0855-416c-8357-dbb15f15dcc8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 150ms :: artifacts dl 6ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [defa

In [2]:
raw_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/raw/transactions')
raw_df_20 = raw_df.orderBy(col("_raw_insert_timestamp").desc()).limit(20)
print(raw_df.count())

54


In [3]:
processed_df = spark.read.format('delta').option("mergeSchema", "true").load('hdfs://spark-test1:9000/processed/transactions')
processed_df_20 = processed_df.orderBy(col("_processed_insert_timestamp").desc()).limit(20)
display(processed_df_20.toPandas())

23/12/12 19:00:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,TransactionId,Amount,CustomerId,TransactionDateTime,TransactionDate,Location,Result,_processed_insert_timestamp,_raw_insert_timestamp
0,19,340.299988,74,2023-12-12 18:58:15,2023-12-12,PARIS,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:19.545439
1,89,539.599976,32,2023-12-12 18:58:19,2023-12-12,SYDNEY,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:23.177651
2,100,344.630005,93,2023-12-12 18:57:58,2023-12-12,TOKYO,SUCCESS,2023-12-12 18:59:06.021106,2023-12-12 18:58:09.870577
3,86,462.549988,54,2023-12-12 18:58:44,2023-12-12,SYDNEY,SUCCESS,2023-12-12 18:59:06.021106,2023-12-12 18:58:48.191607
4,14,672.929993,58,2023-12-12 18:58:04,2023-12-12,TOKYO,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:13.910383
5,51,227.690002,22,2023-12-12 18:58:46,2023-12-12,PARIS,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:50.678288
6,2,880.099976,72,2023-12-12 18:58:06,2023-12-12,PARIS,SUCCESS,2023-12-12 18:59:06.021106,2023-12-12 18:58:13.910383
7,56,870.530029,42,2023-12-12 18:58:19,2023-12-12,PARIS,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:23.177651
8,34,288.200012,33,2023-12-12 18:58:53,2023-12-12,NEW YORK,FAILURE,2023-12-12 18:59:06.021106,2023-12-12 18:58:58.291256
9,59,207.720001,95,2023-12-12 18:58:49,2023-12-12,PARIS,SUCCESS,2023-12-12 18:59:06.021106,2023-12-12 18:58:53.831170
